In [1]:
cd /home/jovyan/GA_DSI/Projects/capstone

/home/jovyan/GA_DSI/Projects/capstone


In [3]:
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import lib.general_utilities as gu

from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression, RidgeClassifier, SGDClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import GridSearchCV, StratifiedShuffleSplit, train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import LabelEncoder

from sklearn.pipeline import make_pipeline

from collections import Counter
import itertools
from wordcloud import WordCloud
import seaborn as sns

from sklearn.metrics import classification_report,precision_recall_curve,confusion_matrix 
from sklearn.metrics import (precision_score,accuracy_score,roc_auc_score,roc_curve, 
                             precision_recall_curve,recall_score,make_scorer,auc)
from tqdm import tqdm

# storing data directories for this notebook.
img_out_dir = 'data/images/Emoticon_NB4/'
data_out_dir = 'data/pickled/Emoticon_NB4/'

%matplotlib inline

In [10]:
# Load full original dataset dataset.
air_df = pd.read_csv("data/csvfiles/kaggle_airline_dataset.csv", encoding='utf-8')

In [11]:
air_df.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [12]:
air_df.shape

(14640, 15)

In [18]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from nltk.stem.porter import PorterStemmer
import emoji

In [19]:
# exclude name of airlines from the tweets. 
exclude = ['virginamerica', 'united', 'southwestair', 'delta', 'usairways',
           'americanair', 'jetblue', 'southwest', 'flight', 'flights',
           'URL', 'AT_USER', 'amp', 'amp;']

In [20]:
# define stop words. Use the defined 'english' stop words. I will add the airline names to that.
stop_words = list(ENGLISH_STOP_WORDS)
stop_words.extend(exclude)

In [23]:
def encode_emojis(text):
    input_txt = text
    for word in text:
        if word in emoji.UNICODE_EMOJI:
            input_txt = input_txt.replace(word, emoji_dict[word])
    return(input_txt)

In [22]:
def extract_emojis(text):
    return(' '.join(word for word in text if word in emoji.UNICODE_EMOJI))

In [24]:
# create a column of only emojis for that text.
air_df['emojis'] = air_df['text'].apply(extract_emojis)

In [30]:
df = air_df[['emojis', 'text']]

In [32]:
df[df.emojis != ''][:10]

,emojis,text
18,❤ ☺ 👍,I ❤️ flying @VirginAmerica. ☺️👍
24,😡,@VirginAmerica you guys messed up my seating.....
30,😢,@VirginAmerica hi! I just bked a cool birthday...
36,💜 ✈,@VirginAmerica Moodlighting is the only way to...
42,🍷 👍 💺 ✈,@VirginAmerica plz help me win my bid upgrade ...
57,😊,@VirginAmerica @ladygaga @carrieunderwood I'm...
62,😍 👌,@VirginAmerica @ladygaga @carrieunderwood all ...
105,💕 💕,"@VirginAmerica - amazing customer service, ag..."
113,🌞 ✈,@VirginAmerica Have a great week 🌞✈
142,😃,@VirginAmerica Can you find us a flt out of LA...


In [33]:
unique_emoji_strings = df.emojis.unique()
unique_emoji_strings

array(['', '❤ ☺ 👍', '😡', '😢', '💜 ✈', '🍷 👍 💺 ✈', '😊', '😍 👌', '💕 💕', '🌞 ✈',
       '😃', '😩 😭', '😎', '🙉', '😁', '❤', '❄ ❄ ❄', '👏', '😂 💗', '🍸', '😒', '👎',
       '👍 👍 ✈ ✈ 💗', '😊 😀 😃 😄', '😘', '👸 💗', '😁 😁 😁', '😍 💗 💗 💗 🎀 🌏 🎀', '😥',
       '🎀 🎀 🎀', '😉', '😍 😍 ✨ 😱 ✈', '✈ 🎉', '💗 🎀 💗', '😃 👍', '🙌 💤', '😞', '♥',
       '👋', '✌', '🙏', '🙌', '💜', '✈', '❄ ❄', '😃 😃 😃', '👿', '😉 😉', '😔', '😭',
       '🙌 🙌', '🙅 🙅 🙅', '😉 😉 😉', '😡 😡', '👏 👏 👏 ✈', '👍', '🆖', '💩', '✔',
       '🌴 🌴', '✅ ❌', '👏 👏 👏 👏 👏 👏', '👏 👏', '😄 😄 😄 😡 😡 😡', '👠 👠 👠', '😜',
       '🎉 🎉 🎉', '😻 😻 😻', '😊 😕', '👌', '👎 👎', '😈', '😡 😡 😡 😡 😤 😤 😤', '❄',
       '👏 👏 👏', '👍 😊', '👍 👌', '😂', '😀', '💪', '😩 😫 😢', '💔 😪', '😕', '😣', '😬',
       '😄', '😂 💁', '😋', '🙌 😏', '😖', '🌟 🌟', '✈ 📱', '👏 👏 🍻 🍻', '😞 😡', '💖',
       '😂 😂 😂 😂 😂 😭 😭 😭 😭 😢 😢 😢 😢', '😅', '😔 😔 😔', '💝', '🚫 ↔', '😏', '😜 😂',
       '😷 😱', '⭐ ✈', '😃 💕 🎵 ✈ ❗ ❤', '😤 🐴', '😍', '😭 😭 😭',
       '😭 😭 💔 💔 💔 💔 💔 💔 💔', '😂 😂', '☺', '💁', '😭 🙏', '😆', '😊 🌴', '👍 👍',
       '✈ ✈', '♥ 🙏', '✈ 😃 👍', '❤ ❤ ❤', '😩', '😑', '💕', '🐩', '😃 💕 

In [34]:
#***** Unique list of emoji features. *****
uni_emoji_feat_lst = unique_emoji_strings.tolist()

In [35]:
uni_emoji_feat_lst[:10]

['', '❤ ☺ 👍', '😡', '😢', '💜 ✈', '🍷 👍 💺 ✈', '😊', '😍 👌', '💕 💕', '🌞 ✈']

## Make a list of unique single emojis found in dataset.

In [36]:
uni_single_emoji=[]
for emoj_str in uni_emoji_feat_lst:
    emoj_lst = emoj_str.split()
    [uni_single_emoji.append(em) for em in emoj_lst if (em not in uni_single_emoji)] 

In [37]:
# The dataset subsetted to where the sentiment confidence level was bove 70 percent, has 17 less
# emojis than the full dataset. I am creating a comprehensive dictionary now so I can run the full
# dataset.
len(uni_single_emoji)

117

In [38]:
#********** unique list of emojis found in corpus. ************
print(uni_single_emoji)

['❤', '☺', '👍', '😡', '😢', '💜', '✈', '🍷', '💺', '😊', '😍', '👌', '💕', '🌞', '😃', '😩', '😭', '😎', '🙉', '😁', '❄', '👏', '😂', '💗', '🍸', '😒', '👎', '😀', '😄', '😘', '👸', '🎀', '🌏', '😥', '😉', '✨', '😱', '🎉', '🙌', '💤', '😞', '♥', '👋', '✌', '🙏', '👿', '😔', '🙅', '🆖', '💩', '✔', '🌴', '✅', '❌', '👠', '😜', '😻', '😕', '😈', '😤', '💪', '😫', '💔', '😪', '😣', '😬', '💁', '😋', '😏', '😖', '🌟', '📱', '🍻', '💖', '😅', '💝', '🚫', '↔', '😷', '⭐', '🎵', '❗', '🐴', '😆', '😑', '🐩', '⤴', '☀', '👊', '💯', '😠', '☕', '📲', '👺', '🙈', '💘', '💙', '👉', '🚪', '😳', '😵', '🚶', '🔵', '😐', '👀', '🍅', '🆘', '⛄', '😓', '🎲', '⌚', '🐳', '⤵', '😮', '😲', '😦', '➡']


In [39]:
# Let me make an emoji dictionary with these unique emojis from my dataset corpus.
cnt = 1
emoji_dict = {}
reverse_lookup_emoji_dict={}

for em in uni_single_emoji:
    val = 'EMOJI_' +str(cnt)
    emoji_dict[em] = val
    reverse_lookup_emoji_dict[val]=em
    cnt+=1

In [29]:
print(emoji_dict)

{'❤': 'EMOJI_1', '☺': 'EMOJI_2', '👍': 'EMOJI_3', '😡': 'EMOJI_4', '😢': 'EMOJI_5', '💜': 'EMOJI_6', '✈': 'EMOJI_7', '🍷': 'EMOJI_8', '💺': 'EMOJI_9', '😊': 'EMOJI_10', '😍': 'EMOJI_11', '👌': 'EMOJI_12', '💕': 'EMOJI_13', '🌞': 'EMOJI_14', '😃': 'EMOJI_15', '😩': 'EMOJI_16', '😭': 'EMOJI_17', '😎': 'EMOJI_18', '🙉': 'EMOJI_19', '😁': 'EMOJI_20', '❄': 'EMOJI_21', '👏': 'EMOJI_22', '😂': 'EMOJI_23', '💗': 'EMOJI_24', '🍸': 'EMOJI_25', '😒': 'EMOJI_26', '👎': 'EMOJI_27', '😀': 'EMOJI_28', '😄': 'EMOJI_29', '😘': 'EMOJI_30', '👸': 'EMOJI_31', '🎀': 'EMOJI_32', '🌏': 'EMOJI_33', '😥': 'EMOJI_34', '😉': 'EMOJI_35', '✨': 'EMOJI_36', '😱': 'EMOJI_37', '🎉': 'EMOJI_38', '🙌': 'EMOJI_39', '💤': 'EMOJI_40', '😞': 'EMOJI_41', '♥': 'EMOJI_42', '👋': 'EMOJI_43', '✌': 'EMOJI_44', '🙏': 'EMOJI_45', '👿': 'EMOJI_46', '😔': 'EMOJI_47', '🙅': 'EMOJI_48', '🆖': 'EMOJI_49', '💩': 'EMOJI_50', '✔': 'EMOJI_51', '🌴': 'EMOJI_52', '✅': 'EMOJI_53', '❌': 'EMOJI_54', '👠': 'EMOJI_55', '😜': 'EMOJI_56', '😻': 'EMOJI_57', '😕': 'EMOJI_58', '😈': 'EMOJI_59', '😤': 

In [30]:
print(reverse_lookup_emoji_dict)

{'EMOJI_1': '❤', 'EMOJI_2': '☺', 'EMOJI_3': '👍', 'EMOJI_4': '😡', 'EMOJI_5': '😢', 'EMOJI_6': '💜', 'EMOJI_7': '✈', 'EMOJI_8': '🍷', 'EMOJI_9': '💺', 'EMOJI_10': '😊', 'EMOJI_11': '😍', 'EMOJI_12': '👌', 'EMOJI_13': '💕', 'EMOJI_14': '🌞', 'EMOJI_15': '😃', 'EMOJI_16': '😩', 'EMOJI_17': '😭', 'EMOJI_18': '😎', 'EMOJI_19': '🙉', 'EMOJI_20': '😁', 'EMOJI_21': '❄', 'EMOJI_22': '👏', 'EMOJI_23': '😂', 'EMOJI_24': '💗', 'EMOJI_25': '🍸', 'EMOJI_26': '😒', 'EMOJI_27': '👎', 'EMOJI_28': '😀', 'EMOJI_29': '😄', 'EMOJI_30': '😘', 'EMOJI_31': '👸', 'EMOJI_32': '🎀', 'EMOJI_33': '🌏', 'EMOJI_34': '😥', 'EMOJI_35': '😉', 'EMOJI_36': '✨', 'EMOJI_37': '😱', 'EMOJI_38': '🎉', 'EMOJI_39': '🙌', 'EMOJI_40': '💤', 'EMOJI_41': '😞', 'EMOJI_42': '♥', 'EMOJI_43': '👋', 'EMOJI_44': '✌', 'EMOJI_45': '🙏', 'EMOJI_46': '👿', 'EMOJI_47': '😔', 'EMOJI_48': '🙅', 'EMOJI_49': '🆖', 'EMOJI_50': '💩', 'EMOJI_51': '✔', 'EMOJI_52': '🌴', 'EMOJI_53': '✅', 'EMOJI_54': '❌', 'EMOJI_55': '👠', 'EMOJI_56': '😜', 'EMOJI_57': '😻', 'EMOJI_58': '😕', 'EMOJI_59': '😈', 'EMOJ

## Pickle the full comprehensive emoji dictionary.

In [31]:
import pickle

In [32]:
filename = data_out_dir+'full_emoji_dict.obj'

In [33]:
with open(filename,'wb') as handle:
    pickle.dump(emoji_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)